In [4]:
import os
import scanpy as sc
import pandas as pd
import numpy as np
import json

In [9]:
from src import sprout_plus
from src import prep_data as prep
import warnings
warnings.filterwarnings("ignore")

# 1. Load files

In [7]:
inputDir = '/public/wanwang6/5.Simpute/2.datasets/4.MERFISH2023/1.input/subset_ST/'
outDir = f'{inputDir}/demo_results/'
sc_exp = pd.read_csv(f'{inputDir}/../SC/SC_exp.tsv',sep = '\t',header=0,index_col=0)
sc_meta = pd.read_csv(f'{inputDir}/../SC/SC_meta.tsv',sep = '\t',header=0,index_col=0)
st_exp = pd.read_csv(f'{inputDir}/ST_exp.tsv',sep = '\t',header=0,index_col=0)
st_coord = pd.read_csv(f'{inputDir}/ST_meta.tsv',sep = '\t',header=0,index_col=0)
st_decon = pd.read_csv(f'{inputDir}/ST_weight.tsv',sep = '\t',header=0,index_col=0)
sc_smurf = pd.read_csv(f'{inputDir}/../SC/subclass_SC_smurf.tsv',sep = '\t',header=0,index_col=0)
lr_df = pd.read_csv('./LR/mouse_LR_pairs.txt',sep='\t',header=None)

# 2.Run sprout

In [1]:
species = 'mouse'
st_tp = 'slide-seq'

In [2]:
# st_coord.index = ['spot_' + str(x) for x in st_coord.index]
# st_exp.index = ['spot_' + str(x) for x in st_exp.index]
# st_decon.index = ['spot_' + str(x) for x in st_decon.index]
# sc_exp.index = ['cell_' + str(x) for x in sc_exp.index]
# sc_meta.index = ['cell_' + str(x) for x in sc_meta.index]
# sc_smurf.index = ['cell_' + str(x) for x in sc_smurf.index]

In [3]:
sc_adata, st_adata, sc_ref, lr_df = prep.prep_all_adata_merfish(sc_exp = sc_exp, st_exp = st_exp, sc_distribution = sc_smurf, 
                                                        sc_meta = sc_meta, st_coord = st_coord, lr_df = lr_df, SP = species)

NameError: name 'prep' is not defined

In [4]:
# sc_ref = sc_smurf[sc_ref.columns]

In [5]:
obj_spex = sprout_plus.SproutImpute(save_path = outDir, st_adata = st_adata, weight = st_decon, 
                 sc_ref = sc_ref, sc_adata = sc_adata, cell_type_key = 'celltype', lr_df = lr_df, 
                 st_tp = st_tp)
obj_spex.prep()
sc_agg_meta = obj_spex.select_cells(use_sc_orig = True, p = 0.1, mean_num_per_spot = 10, mode = 'strict', max_rep = 1, repeat_penalty = 10)

NameError: name 'sprout_plus' is not defined

In [ ]:
# obj_spex.sc_ref = sc_ref

In [ ]:
alpha, beta, gamma, delta, eta = [1, 0.001, 0.001, 0.1, 0.0005]
alter_sc_exp, new_sc_meta = obj_spex.gradient_descent(
                alpha, beta, gamma, delta, eta, 
                init_sc_embed = False,
                iteration = 2, k = 2, W_HVG = 2,
                left_range = 1, right_range = 2, steps = 1, dim = 2)

Running v11 now...
Init sc_coord by affinity embedding...
Avearge shape correlation is: 0.9745196425351544
First-term calculation done!
Second-term calculation done!
By setting k as 6, each spot has average 5.8985107556536125 neighbors.


ValueError: unknown type object

In [ ]:
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)
with open(f'{outDir}/sc_knn.json', 'w') as fp:
    json.dump(obj_spex.sc_knn, fp)
# utils.save_object(obj_spex, f'{outDir}/obj_spex.pkl')
if st_tp != 'slide-seq':
    new_coord = new_sc_meta[['adj_spex_UMAP1','adj_spex_UMAP2']]
    new_coord.to_csv(f'{outDir}/spot_coord_best.tsv',sep = '\t',header=True,index=True)

***

In [ ]:
from src import optimizers
from src.cell_selection import *

In [ ]:
picked_time = obj_spex.picked_time

In [ ]:
new_sc_meta

,sc_id,spot,exp_cor_before,interface_cor_before,mix_cor_before,exp_cor_after,interface_cor_after,mix_cor_after,celltype,st_x,st_y,spex_UMAP1,spex_UMAP2,indice,core1,core2,centering_scale_factor,adj_spex_UMAP1,adj_spex_UMAP2
0,cell_1078,spot_8,0.972740,0.911212,0.966587,0.982461,0.927312,0.911523,ExcitatoryL5,375,6375,0.527846,7.280060,167,0.342356,7.428467,1578.597672,667.814384,6140.725083
1,cell_703,spot_8,0.972740,0.911212,0.966587,0.982461,0.927312,0.911523,ExcitatoryL6,375,6375,0.210699,7.492771,167,0.342356,7.428467,1578.597672,167.167783,6476.510700
2,cell_62,spot_8,0.972740,0.911212,0.966587,0.982461,0.927312,0.911523,ExcitatoryL6,375,6375,0.221804,7.410657,167,0.342356,7.428467,1578.597672,184.698252,6346.885384
3,cell_824,spot_8,0.972740,0.911212,0.966587,0.982461,0.927312,0.911523,ExcitatoryL6,375,6375,0.332700,7.578509,167,0.342356,7.428467,1578.597672,359.757371,6611.856042
4,cell_264,spot_8,0.972740,0.911212,0.966587,0.982461,0.927312,0.911523,Olig,375,6375,0.320366,7.569242,167,0.342356,7.428467,1578.597672,340.287623,6597.227409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1796,cell_1403,spot_3,0.977388,0.970038,0.976653,0.980615,0.982895,0.915159,Olig,375,2625,-2.008883,9.260750,112,-1.914588,9.232329,420.647405,335.335393,2636.954988
1797,cell_341,spot_3,0.977388,0.970038,0.976653,0.980615,0.982895,0.915159,Olig,375,2625,-1.559768,8.999914,112,-1.914588,9.232329,420.647405,524.254168,2527.235149
1798,cell_486,spot_3,0.977388,0.970038,0.976653,0.980615,0.982895,0.915159,Olig,375,2625,-2.665015,9.713579,112,-1.914588,9.232329,420.647405,59.334877,2827.436484
1799,cell_1321,spot_3,0.977388,0.970038,0.976653,0.980615,0.982895,0.915159,Astro,375,2625,-1.852528,9.191294,112,-1.914588,9.232329,420.647405,401.105678,2607.738459


In [ ]:
new_sc_meta['exp_cor_before'].mean()

0.9805432405392891

In [ ]:
np.mean(new_sc_meta['exp_cor_after'])

0.9819837519731657

In [ ]:
# TODO st data是乱序的只有st coord 是正序 没关系吗？

In [ ]:
st_exp = obj_spex.st_exp
st_coord = obj_spex.st_coord
st_aff_profile_df = obj_spex.st_aff_profile_df
sc_exp = obj_spex.sc_exp
sc_meta = obj_spex.sc_meta
sum_sc_agg_exp = obj_spex.sum_sc_agg_exp
sc_agg_aff_profile_df = obj_spex.sc_agg_aff_profile_df
init_sc_df = obj_spex.init_sc_df
picked_time = obj_spex.picked_time
lr_df = obj_spex.lr_df
st_tp = obj_spex.st_tp
p = 0.1
T_HALF = 10

In [ ]:
st_dist = optimizers.findSpotNeighbor(st_coord,st_tp)
# dict like: 'spot_0': ['spot_1', 'spot_19']
spots_nn_lst = dict(st_dist.apply(lambda x: x.index[x == 1].tolist(), axis=1))
# dict like: 'tp1':['cell_0','cell_1']
tp_idx_dict = get_tp_idx_dict(sc_meta)
new_spot_cell_dict = {}
spot_idx_lst = list(st_exp.index)
result = pd.DataFrame()

In [ ]:
for spot in spot_idx_lst:
    '''
    s_: spot exp of st_exp or sc_agg
    _i: indice numerical index of cell_id or spot
    '''
    ########## ST ##########
    # Transform to numerical spot id, subset from csr_matrix
    s_exp = st_exp.loc[spot]
    norm_s_exp = s_exp/np.std(s_exp)
    ########## SC ########## 
    s_sc_agg_sum = sum_sc_agg_exp.loc[spot]
    norm_s_sc_agg_sum = s_sc_agg_sum/np.std(s_sc_agg_sum)
    # generate baseline corr
    max_exp_cor = np.corrcoef(norm_s_exp,norm_s_sc_agg_sum)[0][1]
    # print(f'Baseline cor of spot {spot} is {max_exp_cor}')
    ###### Interface ########
    nn_spot = spots_nn_lst[spot]
    a_ss = st_aff_profile_df.loc[(spot,nn_spot),:]
    a_cc = sc_agg_aff_profile_df.loc[(spot,nn_spot),:]
    corr = np.diag(np.corrcoef(a_ss, a_cc)[:a_ss.shape[0], a_ss.shape[0]:])
    max_aff_cor = corr.mean()
    # print(f'Baseline cor of interface {spot} is {max_aff_cor}')
    max_mix_cor = max_exp_cor*(1-p) + max_aff_cor*p
    # print(f'Baseline mix cor of {spot} is {max_mix_cor}')
    # for each cell in spot
    spot_cell_lst = init_sc_df[init_sc_df['spot'] == spot]['sc_id'].tolist()
    # print(f'orig spot_cell_lst {spot_cell_lst}')
    for i in range(len(spot_cell_lst)):
        cell = spot_cell_lst[i]
        # print(cell)
        spot_cell_lst.remove(cell)
        # print(spot_cell_lst)
        # calculate remain agg exp
        spot_remain_mat = sc_exp.loc[spot_cell_lst]
        remain_exp = np.sum(spot_remain_mat)
        # get candidate cells from the same type
        removed_type = sc_meta.loc[cell]['celltype']
        candi_cell_id = tp_idx_dict[removed_type]
        candi_exp = sc_exp.loc[candi_cell_id]
        # calculate replaced agg for each candidates
        candi_exp_sum = candi_exp + remain_exp
        # [exp cor]
        exp_candi_cor = candi_exp_sum.T.corrwith(s_exp)
        # [interface cor]
        interface_candi_cor = pd.DataFrame()
        for nn_s in nn_spot:
            # nn_s: the neighbor of spot
            a_sn = a_ss.loc[(spot,nn_s)]
            # nn_s_agg_exp: summed exp of init selected cells of nn_s
            nn_s_agg_exp = sum_sc_agg_exp.loc[nn_s]
            # interface between candidate cells and its neighbors regarding aggregated sum
            candi_aff_profile = cal_sc_candi_aff_profile(nn_s_agg_exp, candi_exp_sum, lr_df)
            interface_candi_tmp = candi_aff_profile.T.corrwith(a_sn)
            interface_candi_tmp = pd.DataFrame(interface_candi_tmp,columns=[nn_s])
            interface_candi_cor = pd.concat((interface_candi_cor,interface_candi_tmp),axis = 1)
    #     break
    # break
        interface_candi_cor['mean'] = interface_candi_cor.mean(axis = 1)
        # print(interface_candi_cor.head(5).index)
        # print(exp_candi_cor.head(5).index)
        # TODO del
        prob = half_life_prob(picked_time['count'].values,T_HALF)
        picked_time['prob'] = prob
        cor_df = interface_candi_cor.loc[exp_candi_cor.index,'mean']*p + (1-p)*exp_candi_cor
        adj_cor_df = picked_time.loc[cor_df.index,'prob'] * cor_df
        max_idx = adj_cor_df.idxmax()
        mix_corr = adj_cor_df.loc[adj_cor_df.idxmax()]
        spot_cell_lst.insert(0,max_idx)
        exp_cor = exp_candi_cor.loc[max_idx]
        inter_cor = interface_candi_cor['mean'].loc[max_idx]
        # update cell picked time
        picked_time.loc[cell,'count'] -= 1
        picked_time.loc[max_idx,'count'] += 1
        # print(f'  Change cell {cell} exp_cor is {exp_cor}; inter_cor is {inter_cor}; mix cor of {spot} is {mix_corr}')
        break


In [ ]:
adj_cor_df = picked_time.loc[cor_df.index,'prob'] * cor_df
max_idx = adj_cor_df.idxmax()
mix_corr = adj_cor_df.loc[adj_cor_df.idxmax()]
spot_cell_lst.insert(0,max_idx)
exp_cor = exp_candi_cor.loc[max_idx]
inter_cor = interface_candi_cor['mean'].loc[max_idx]
picked_time.loc[max_idx,]

count    1.0
prob     1.0
Name: cell_1037, dtype: float64

In [ ]:
picked_time

,count,prob
cell_0,1.0,0.933033
cell_1,1.0,0.933033
cell_2,1.0,0.933033
cell_3,1.0,0.933033
cell_4,1.0,0.933033
...,...,...
cell_1518,1.0,0.933033
cell_1519,1.0,0.933033
cell_1520,1.0,0.933033
cell_1521,1.0,0.933033


In [ ]:
picked_time.loc[max_idx,'count'] += 1

In [ ]:
    tmp = pd.DataFrame(spot_cell_lst,columns = ['sc_id'])
    tmp['spot'] = spot
    tmp['exp_cor_before'] = max_exp_cor
    tmp['interface_cor_before'] = max_aff_cor
    tmp['mix_cor_before'] = max_mix_cor
    tmp['exp_cor_after'] = exp_cor
    tmp['interface_cor_after']= inter_cor
    tmp['mix_cor_after'] = mix_corr
    result = pd.concat((result,tmp))
    new_spot_cell_dict[spot] = spot_cell_lst
    # break
result['celltype'] = sc_meta.loc[result['sc_id']]['celltype'].values
result.index = range(len(result))
result.index = result.index.map(str)

In [ ]:
def reselect_cell(st_exp, st_coord, st_aff_profile_df, sc_exp, sc_meta, sum_sc_agg_exp, sc_agg_aff_profile_df, init_sc_df, lr_df, st_tp, p = 0.1):
    '''
    Reselect cells from sc exp data for higher exp and interface correlation
    No repeat
    Runtime: 20s for each spot with 10 cells; 2s each cell.
    st_exp = obj_spex.st_exp
    st_coord = obj_spex.st_coord
    st_aff_profile_df = obj_spex.st_aff_profile_df
    sc_exp = obj_spex.sc_exp
    sc_meta = obj_spex.sc_meta
    sum_sc_agg_exp = obj_spex.sum_sc_agg_exp
    sc_agg_aff_profile_df = obj_spex.sc_agg_aff_profile_df
    init_sc_df = obj_spex.init_sc_df
    lr_df = obj_spex.lr_df
    st_tp = obj_spex.st_tp
    p = 0.1
    '''
    st_dist = optimizers.findSpotNeighbor(st_coord,st_tp)
    spots_nn_lst = dict(st_dist.apply(lambda x: x.index[x == 1].tolist(), axis=1))
    # 
    tp_idx_dict = get_tp_idx_dict(sc_meta)
    new_spot_cell_dict = {}
    spot_idx_lst = list(st_exp.index)
    result = pd.DataFrame()
    for spot in spot_idx_lst:
        '''
        s_: spot exp of st_exp or sc_agg
        _i: indice numerical index of cell_id or spot
        '''
        # print(f'reselect cells for spot {spot}')
        ########## ST ##########
        # Transform to numerical spot id, subset from csr_matrix
        s_exp = st_exp.loc[spot]
        norm_s_exp = s_exp/np.std(s_exp)
        ########## SC ########## 
        s_sc_agg_sum = sum_sc_agg_exp.loc[spot]
        norm_s_sc_agg_sum = s_sc_agg_sum/np.std(s_sc_agg_sum)
        # generate baseline corr
        max_exp_cor = np.corrcoef(norm_s_exp,norm_s_sc_agg_sum)[0][1]
        # print(f'Baseline cor of spot {spot} is {max_exp_cor}')
        ###### Interface ########
        nn_spot = spots_nn_lst[spot]
        a_ss = st_aff_profile_df.loc[(spot,nn_spot),:]
        a_cc = sc_agg_aff_profile_df.loc[(spot,nn_spot),:]
        corr = np.diag(np.corrcoef(a_ss, a_cc)[:a_ss.shape[0], a_ss.shape[0]:])
        max_aff_cor = corr.mean()
        # print(f'Baseline cor of interface {spot} is {max_aff_cor}')
        max_mix_cor = max_exp_cor*(1-p) + max_aff_cor*p
        # print(f'Baseline mix cor of {spot} is {max_mix_cor}')
        # for each cell in spot
        spot_cell_lst = init_sc_df[init_sc_df['spot'] == spot]['sc_id'].tolist()
        # print(f'orig spot_cell_lst {spot_cell_lst}')
        for i in range(len(spot_cell_lst)):
            cell = spot_cell_lst[i]
            # print(cell)
            spot_cell_lst.remove(cell)
            # print(spot_cell_lst)
            # calculate remain agg exp
            spot_remain_mat = sc_exp.loc[spot_cell_lst]
            remain_exp = np.sum(spot_remain_mat)
            # get candidate cells from the same type
            removed_type = sc_meta.loc[cell]['celltype']
            candi_cell_id = tp_idx_dict[removed_type]
            candi_exp = sc_exp.loc[candi_cell_id]
            # calculate replaced agg for each candidates
            candi_exp_sum = candi_exp + remain_exp
            # [exp cor]
            exp_candi_cor = candi_exp_sum.T.corrwith(s_exp)
            # [interface cor]
            interface_candi_cor = pd.DataFrame()
            for nn_s in nn_spot:
                # nn_s: the neighbor of spot
                a_sn = a_ss.loc[(spot,nn_s)]
                # nn_s_agg_exp: summed exp of init selected cells of nn_s
                nn_s_agg_exp = sum_sc_agg_exp.loc[nn_s]
                # interface between candidate cells and its neighbors regarding aggregated sum
                candi_aff_profile = cal_sc_candi_aff_profile(nn_s_agg_exp, candi_exp_sum, lr_df)
                interface_candi_tmp = candi_aff_profile.T.corrwith(a_sn)
                interface_candi_tmp = pd.DataFrame(interface_candi_tmp,columns=[nn_s])
                interface_candi_cor = pd.concat((interface_candi_cor,interface_candi_tmp),axis = 1)
            interface_candi_cor['mean'] = interface_candi_cor.mean(axis = 1)
            # print(interface_candi_cor.head(5).index)
            # print(exp_candi_cor.head(5).index)
            # TODO del
            cor_df = interface_candi_cor.loc[exp_candi_cor.index,'mean']*p + (1-p)*exp_candi_cor
            max_idx = cor_df.idxmax()
            mix_corr = cor_df.loc[cor_df.idxmax()]
            spot_cell_lst.insert(0,max_idx)
            exp_cor = exp_candi_cor.loc[max_idx]
            inter_cor = interface_candi_cor['mean'].loc[max_idx]
            # print(f'  Change cell {cell} exp_cor is {exp_cor}; inter_cor is {inter_cor}; mix cor of {spot} is {mix_corr}')
            # break
        tmp = pd.DataFrame(spot_cell_lst,columns = ['sc_id'])
        tmp['spot'] = spot
        tmp['exp_cor_before'] = max_exp_cor
        tmp['interface_cor_before'] = max_aff_cor
        tmp['mix_cor_before'] = max_mix_cor
        tmp['exp_cor_after'] = exp_cor
        tmp['interface_cor_after']= inter_cor
        tmp['mix_cor_after'] = mix_corr
        result = pd.concat((result,tmp))
        new_spot_cell_dict[spot] = spot_cell_lst
        # break
    result['celltype'] = sc_meta.loc[result['sc_id']]['celltype'].values
    result.index = range(len(result))
    result.index = result.index.map(str)
    return result


In [ ]:
st_exp = obj_spex.st_exp
sc_knn = obj_spex.sc_knn 
st_aff_profile_df = obj_spex.st_aff_profile_df
alter_sc_exp = obj_spex.alter_sc_exp
sc_meta = obj_spex.sc_agg_meta
spot_cell_dict = obj_spex.spot_cell_dict
lr_df = obj_spex.lr_df

In [ ]:
term4_df, loss_4 = cal_term4(st_exp,sc_knn,st_aff_profile_df,alter_sc_exp,sc_meta,spot_cell_dict,lr_df)

NameError: name 'cal_term4' is not defined

In [ ]:
obj_spex.ALPHA,obj_spex.BETA,obj_spex.GAMMA,obj_spex.DELTA

(0.020253682220164052,
 0.00025403830843444877,
 0.0008631145032363329,
 0.0018113246793395382)

In [ ]:
obj_spex.loss2

2881.439617870177

In [ ]:
def cal_term4(st_exp,sc_knn,st_aff_profile_df,alter_sc_exp,sc_meta,spot_cell_dict,lr_df):
    ''' 
    st_exp = obj_spex.st_exp
    sc_knn = obj_spex.sc_knn 
    st_aff_profile_df = obj_spex.st_aff_profile_df
    alter_sc_exp = obj_spex.alter_sc_exp
    sc_meta = obj_spex.sc_meta
    spot_cell_dict = obj_spex.spot_cell_dict
    lr_df = obj_spex.lr_df
    '''
    # generate knn_df: cell_idx	-> nn_cell_idx
    knn_df = pd.DataFrame(sc_knn.items(), columns=['cell_idx', 'nn_cell_idx'])
    knn_df = knn_df.explode('nn_cell_idx')
    nn_cell_idx = knn_df['nn_cell_idx'].tolist()
    df = sc_meta.loc[nn_cell_idx].copy()
    knn_df['spot'] = df['spot'].values
    term4_LR = pd.DataFrame()
    loss_4 = 0
    n_knn = 0
    #print(knn_df)
    for spot in st_exp.index:
        #print(spot)
        spot_cells = spot_cell_dict[spot]
        # 1. find knn id and its affiliated spot
        spot_knn_df = knn_df[knn_df['cell_idx'].isin(spot_cells)]
        cell_idx = spot_knn_df['cell_idx']
        cell_nn_idx = spot_knn_df['nn_cell_idx']
        n_knn += len(cell_nn_idx)
        #print(spot_knn_df)
        # some spot has no nn for any cell in it.
        if cell_nn_idx.tolist():
            # 2. calculate acc
            tmp_acc = cal_sc_aff_profile(cell_idx, cell_nn_idx, alter_sc_exp, lr_df)
            tmp_acc = tmp_acc.reset_index()
            del tmp_acc[tmp_acc.columns[0]]
            tmp_acc['spot'] = spot_knn_df['spot'].values
            a_cc = tmp_acc.groupby('spot').mean()
            # 3. calculate ass
            a_ss = st_aff_profile_df.loc[(spot,a_cc.index.tolist()),:]
            a_cc_modi = np.sqrt(a_cc/2)
            a_ss_modi = np.sqrt(a_ss/2)
            res_tmp = a_cc_modi - a_ss_modi.values
            loss_tmp = np.sum((res_tmp**2).values)
            loss_4 += loss_tmp
            if np.isnan(loss_tmp):
                print(f'{spot}')
                print(f'acc{a_cc}')
                print(f'ass{a_ss}')
                print(f'a_cc_modi{a_cc_modi}')
                print(f'a_ss_modi{a_ss_modi}')
                print(f'res_tmp{res_tmp}')
            # 4. calculate multiplier
            sum_r = calSumNNRL(alter_sc_exp, spot_knn_df, cell_nn_idx, lr_df[1])
            sum_l = calSumNNRL(alter_sc_exp, spot_knn_df, cell_nn_idx, lr_df[0])
            res_L = sum_r.groupby('cell_idx').apply(multiply_spots,res_tmp = res_tmp)
            res_R = sum_l.groupby('cell_idx').apply(multiply_spots,res_tmp = res_tmp)
            # ? res_R.columns = sum_r.columns
            res_LR = pd.concat([res_L,res_R],axis =1)
            # sum for each adj spot
            ave_res_LR = res_LR.groupby('cell_idx').sum()
            # ave for same gene in LRdb
            res = ave_res_LR.T.groupby('symbol').mean().T
            term4_LR = pd.concat([term4_LR,res],axis =0)
        #break
    loss_4 /= n_knn
    if np.isnan(loss_4):
        print('nananananana')
    term4_df = complete_other_genes(alter_sc_exp, term4_LR)
    return term4_df, loss_4


In [ ]:
def cal_st_cor(st_exp, sum_sc_agg_exp, spot):
    s_exp = st_exp.loc[spot]
    norm_s_exp = s_exp/np.std(s_exp)
    ########## SC ########## 
    norm_s_sc_agg_sum = sum_sc_agg_exp/np.std(sum_sc_agg_exp)
    # generate baseline corr
    max_exp_cor = np.corrcoef(norm_s_exp,norm_s_sc_agg_sum)[0][1]
    print(max_exp_cor)
    return max_exp_cor

In [ ]:
def reselect_cell(st_exp, st_coord, st_aff_profile_df, sc_exp, sc_meta, sum_sc_agg_exp, sc_agg_aff_profile_df, init_sc_df, lr_df, st_tp, p = 0.1):
    '''
    Reselect cells from sc exp data for higher exp and interface correlation
    No repeat
    Runtime: 20s for each spot with 10 cells; 2s each cell.
    '''
    st_dist = optimizers.findSpotNeighbor(st_coord,st_tp)
    spots_nn_lst = dict(st_dist.apply(lambda x: x.index[x == 1].tolist(), axis=1))
    # 
    tp_idx_dict = get_tp_idx_dict(sc_meta)
    new_spot_cell_dict = {}
    spot_idx_lst = list(st_exp.index)
    result = pd.DataFrame()
    print('Selection start...')
    for spot in spot_idx_lst:
        '''
        s_: spot exp of st_exp or sc_agg
        _i: indice numerical index of cell_id or spot
        '''
        # print(f'reselect cells for spot {spot}')
        ########## ST ##########
        # Transform to numerical spot id, subset from csr_matrix
        s_exp = st_exp.loc[spot]
        norm_s_exp = s_exp/np.std(s_exp)
        ########## SC ########## 
        s_sc_agg_sum = sum_sc_agg_exp.loc[spot]
        norm_s_sc_agg_sum = s_sc_agg_sum/np.std(s_sc_agg_sum)
        # generate baseline corr
        max_exp_cor = np.corrcoef(norm_s_exp,norm_s_sc_agg_sum)[0][1]
        # print(f'Baseline cor of spot {spot} is {max_exp_cor}')
        ###### Interface ########
        nn_spot = spots_nn_lst[spot]
        a_ss = st_aff_profile_df.loc[(spot,nn_spot),:]
        a_cc = sc_agg_aff_profile_df.loc[(spot,nn_spot),:]
        corr = np.diag(np.corrcoef(a_ss, a_cc)[:a_ss.shape[0], a_ss.shape[0]:])
        max_aff_cor = corr.mean()
        # print(f'Baseline cor of interface {spot} is {max_aff_cor}')
        max_mix_cor = max_exp_cor*(1-p) + max_aff_cor*p
        # print(f'Baseline mix cor of {spot} is {max_mix_cor}')
        # for each cell in spot
        spot_cell_lst = init_sc_df[init_sc_df['spot'] == spot]['sc_id'].tolist()
        # print(f'orig spot_cell_lst {spot_cell_lst}')
        for i in range(len(spot_cell_lst)):
            cell = spot_cell_lst[i]
            # print(cell)
            spot_cell_lst.remove(cell)
            # print(spot_cell_lst)
            # calculate remain agg exp
            spot_remain_mat = sc_exp.loc[spot_cell_lst]
            remain_exp = np.sum(spot_remain_mat)
            # get candidate cells from the same type
            removed_type = sc_meta.loc[cell]['celltype']
            candi_cell_id = tp_idx_dict[removed_type]
            candi_exp = sc_exp.loc[candi_cell_id]
            # calculate replaced agg for each candidates
            candi_exp_sum = candi_exp + remain_exp
            # [exp cor]
            exp_candi_cor = candi_exp_sum.T.corrwith(s_exp)
            # [interface cor]
            interface_candi_cor = pd.DataFrame()
            for nn_s in nn_spot:
                # nn_s: the neighbor of spot
                a_sn = a_ss.loc[(spot,nn_s)]
                # nn_s_agg_exp: summed exp of init selected cells of nn_s
                nn_s_agg_exp = sum_sc_agg_exp.loc[nn_s]
                # interface between candidate cells and its neighbors regarding aggregated sum
                candi_aff_profile = cal_sc_candi_aff_profile(nn_s_agg_exp, candi_exp_sum, lr_df)
                interface_candi_tmp = candi_aff_profile.T.corrwith(a_sn)
                interface_candi_tmp = pd.DataFrame(interface_candi_tmp,columns=[nn_s])
                interface_candi_cor = pd.concat((interface_candi_cor,interface_candi_tmp),axis = 1)
            interface_candi_cor['mean'] = interface_candi_cor.mean(axis = 1)
            # print(interface_candi_cor.head(5).index)
            # print(exp_candi_cor.head(5).index)
            # TODO del
            cor_df = interface_candi_cor.loc[exp_candi_cor.index,'mean']*p + (1-p)*exp_candi_cor
            max_idx = cor_df.idxmax()
            mix_corr = cor_df.loc[cor_df.idxmax()]
            spot_cell_lst.insert(0,max_idx)
            exp_cor = exp_candi_cor.loc[max_idx]
            inter_cor = interface_candi_cor['mean'].loc[max_idx]
            # print(f'  Change cell {cell} exp_cor is {exp_cor}; inter_cor is {inter_cor}; mix cor of {spot} is {mix_corr}')
            # break
        tmp = pd.DataFrame(spot_cell_lst,columns = ['sc_id'])
        tmp['spot'] = spot
        tmp['exp_cor_before'] = max_exp_cor
        tmp['interface_cor_before'] = max_aff_cor
        tmp['mix_cor_before'] = max_mix_cor
        tmp['exp_cor_after'] = exp_cor
        tmp['interface_cor_after']= inter_cor
        tmp['mix_cor_after'] = mix_corr
        result = pd.concat((result,tmp))
        new_spot_cell_dict[spot] = spot_cell_lst
        # break
    result['celltype'] = sc_meta.loc[result['sc_id']]['celltype'].values
    return new_spot_cell_dict, result